In [ ]:
import os
import numpy as np
import random
import glob
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.ticker import MaxNLocator
%matplotlib inline

import time
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras import backend as K
from tensorflow.keras.applications import *
from tensorflow.keras import *


import seaborn as sns
import IPython.display as ipd
from IPython.core.display import display, clear_output


from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

from collections import defaultdict

from tqdm.notebook import tqdm
import xml.etree.ElementTree as ET

import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

!pip install -q -U tensorflow-addons
import tensorflow_addons as tfa

     |████████████████████████████████| 1.1 MB 5.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if not os.path.exists('./IAM Dataset'):
    ret = os.system("cp '/content/drive/MyDrive/Dataset/IAM_line_images.zip' '/content/IAM_Handwritten_images.zip'")
    ret = os.system("cp '/content/drive/MyDrive/Dataset/IAM_line_labels.zip' '/content/IAM_Handwritten_labels.zip'")

    if ret == 0:  print("Data loaded")
    os.system("unzip -qq '/content/IAM_Handwritten_images.zip'")
    os.system("unzip -qq '/content/IAM_Handwritten_labels.zip'")
    print('unziped')

    !rm IAM_Handwritten_images.zip
    !rm IAM_Handwritten_labels.zip
    print('Zip removed')



Data loaded
unziped
Zip removed


In [ ]:
import cv2
import numpy as np
# For image drawing
from PIL import Image, ImageDraw, ImageFont, ImageEnhance

def change_contrast(img, level):
    # Input is a Image type
    # use Image.fromarray() on numpy
    factor = (259 * (level + 255)) / (255 * (259 - level))
    def contrast(c):
        return 128 + factor * (c - 128)
    return img.point(contrast)


def change_sharpness(img, level):
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(level)
    return img


def adjust(img, blevel, slevel, clevel, colevel):
    # brightness
    benhance = ImageEnhance.Brightness(img)
    img = benhance.enhance(blevel)
    # sharpness
    img = change_sharpness(img, slevel)
    # contrast
    img = change_contrast(img, clevel)
    # color
    cenhance = ImageEnhance.Color(img)
    img = cenhance.enhance(colevel)
    return img


def improveImage(img_dir):
    img = cv2.imread(img_dir, -1)
    rgb_planes = cv2.split(img)

    result_planes = []
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img, None, alpha=0,
                                 beta=255, norm_type=cv2.NORM_MINMAX,
                                 dtype=cv2.CV_8UC1)

        result_planes.append(diff_img)
        result_norm_planes.append(norm_img)

    result = cv2.merge(result_planes)
    result_norm = cv2.merge(result_norm_planes)
    return result_norm


def convert_image(image_dir):
    img = improveImage(image_dir)
    img = adjust(Image.fromarray(img),
                  blevel=0.7, slevel=1, clevel=255,
                  colevel=1)
    return img

In [ ]:
from collections import defaultdict
d={}
labels_path = '/content/IAM_line labels/*'
check = {}
count = defaultdict(int)
for filename in sorted(glob.glob(labels_path)):
    tree = ET.parse(filename)
    root = tree.getroot()
    passage_id = root.attrib.get('id')
    check[root.attrib.get('writer-id')]=check.get(root.attrib.get('writer-id'),0)+1


for filename in sorted(glob.glob(labels_path)):
    tree = ET.parse(filename)
    root = tree.getroot()
    if count[root.attrib.get('writer-id')]<1:
   # if check[root.attrib.get('writer-id')]>=2 and count[root.attrib.get('writer-id')]<2:
            count[root.attrib.get('writer-id')] = count.get(root.attrib.get('writer-id'),0)+1
            for i in root.iter('line'):
                d[i.attrib.get('id')] = root.attrib.get('writer-id')

print(check)

image_files = []
ground_files = []
images = sorted(glob.glob('/content/IAM_line images/**/*.png', recursive=True))



for filename in sorted(tqdm(images)):
    s_filename = filename
    s_filename = s_filename.split('/')[-1]
    s_filename = s_filename.split('.')[0]

    try:
        im = Image.open(filename)
        if s_filename in d:
            # im = convert_image(filename)
            # im.save(filename,format='png')
            image_files.append(filename)
            ground_files.append(d[s_filename])
    except:
      continue

image_files = np.asarray(image_files)
ground_files = np.asarray(ground_files)
ground_files = LabelEncoder().fit_transform(ground_files)
print(image_files.shape,ground_files.shape)
print(np.unique(image_files).shape,np.unique(ground_files).shape)
print(np.unique(ground_files))

{'000': 59, '001': 2, '002': 1, '003': 2, '004': 1, '005': 2, '006': 1, '007': 2, '008': 2, '009': 2, '010': 2, '011': 2, '012': 2, '013': 3, '014': 2, '015': 1, '016': 3, '017': 3, '018': 2, '019': 3, '020': 1, '021': 1, '022': 1, '023': 1, '024': 1, '025': 5, '026': 5, '027': 1, '028': 1, '029': 1, '030': 1, '031': 1, '032': 1, '033': 1, '034': 1, '035': 1, '036': 1, '037': 5, '038': 1, '039': 1, '040': 1, '041': 1, '042': 1, '043': 1, '044': 1, '045': 1, '046': 1, '047': 1, '048': 1, '049': 1, '050': 1, '051': 1, '052': 1, '053': 1, '054': 1, '055': 1, '056': 1, '058': 4, '059': 4, '060': 4, '061': 4, '062': 3, '063': 3, '064': 4, '065': 2, '066': 2, '067': 1, '068': 1, '069': 1, '070': 1, '071': 1, '072': 1, '073': 1, '074': 1, '075': 1, '076': 1, '077': 1, '078': 1, '079': 1, '080': 1, '081': 2, '082': 1, '083': 1, '084': 2, '085': 6, '086': 1, '087': 3, '088': 3, '089': 3, '090': 3, '091': 2, '092': 3, '093': 3, '094': 3, '095': 2, '096': 1, '097': 1, '098': 1, '099': 1, '100': 1

  0%|          | 0/13353 [00:00<?, ?it/s]

(5660,) (5660,)
(5660,) (657,)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238

In [ ]:
writer = 25
purity = 0
num_classes_per_batch=16
num_samples_per_class=4

In [ ]:
l = np.zeros((657,),dtype='int32')
x = []
gy = []

for i in range(image_files.shape[0]):
  if ground_files[i]<writer:
      x.append(image_files[i])
      gy.append(ground_files[i])
      im = convert_image(image_files[i])
      im.save(image_files[i],format='png')


x = np.asarray(x)
gy = np.asarray(gy)
gy = LabelEncoder().fit_transform(gy)
print(x.shape,gy.shape)
print(np.unique(x).shape,np.unique(gy).shape)

(231,) (231,)
(231,) (25,)


In [ ]:
def pseudo_label_assign(purity,gy):

    count = np.unique(gy,return_index=True,return_counts=True)
    counter = np.array(count[2])
    check = np.ones((writer),dtype=np.int8)
    py_order = []
    py =[]
    p=0
    if purity == 0:
        for i in range(gy.shape[0]):
          py.append(p)
          p+=1
        py = np.asarray(py,dtype=np.int16)
        py = LabelEncoder().fit_transform(py)
        return py
    for i in range(counter.shape[0]):

        res = int(np.floor(counter[i]/(1/purity)))
        remainder = int(counter[i]%(1/purity))
        order = []
        for k in range(int(1/purity)):
          order.append(res)
        for j in range(remainder):
            order[j] +=1
        py_order.append(order)

    p=0
    p_order =[]
    for i in py_order:
        for j in i:
            p_order.append(j)

    for i in range(len(p_order)):
        for j in range(p_order[i]):
            py.append(p)
        p+=1
    py = np.asarray(py,dtype=np.int16)
    py = LabelEncoder().fit_transform(py)

    return py

py= pseudo_label_assign(purity=purity,gy=gy)

print(x.shape,py.shape,gy.shape)
print(np.unique(x).shape,np.unique(py).shape,np.unique(gy).shape)

(231,) (231,) (231,)
(231,) (231,) (25,)


In [ ]:
import math
img = []
pse_label=[]
grd_label = []
input_shape = (112,224,1)
def image_resize(image, width = None, height = None):
        """Img is a PIL image"""
        dim = None
        (w, h) = image.size

        if width is None and height is None:
            return None
        if width is None:
            r = height / float(h)
            dim = (int(w * r), height)
        else:
            r = width / float(w)
            dim = (width, int(h * r))

        return image.resize(dim)

for i in range(x.shape[0]):
    im = Image.open(x[i])
    im = image_resize(im,height=input_shape[0])
    im = np.asarray(im,dtype=np.float32)
    height,width = im.shape
    # print(im.shape)
    end_pos = 0
    while end_pos < width-224:
        tmp_img = im[:,end_pos:end_pos+224]
        end_pos = end_pos+224
        tmp = np.ones((112,224,3),dtype=np.float32)
        tmp[...,0]=tmp_img
        tmp[...,1]=tmp_img
        tmp[...,2]=tmp_img
        img.append(tmp)
        pse_label.append(py[i])
        grd_label.append(gy[i])

# img = img[:,:,:,np.newaxis]
X = np.asarray(img,dtype=np.uint8)
py = np.asarray(pse_label,dtype=np.int16)
gy = np.asarray(grd_label,dtype=np.int16)
# x = x[:,:,:,np.newaxis]
print(X.shape,py.shape,gy.shape)
print(np.unique(x,axis=0).shape,np.unique(py).shape,np.unique(gy).shape)

(1401, 112, 224, 3) (1401,) (1401,)
(231,) (231,) (25,)


In [ ]:
import shutil

shutil.rmtree('/content/IAM_line images')
shutil.rmtree('/content/IAM_line labels')

In [ ]:
class AEGenerator(tf.keras.utils.Sequence):
    """ Recieves X and y
        Performes pairwise matching with a batch size
        both pairs are generated from the input X and y

        imgs         : Input image directory
        y            : Pseudo label, Numpy Array, Output shape=(bs, ) or (bs, 1)
        gt           : Ground truth of the actual class, Numpy Array, Output shape=(bs, ) or (bs, 1).
        dist      : Distance parameter for AE
        noise     : Ratio of augmentation
        scale     : The maximum limit of noise that would be mixed with X
        show_logs : Show random selection errors

                    Use only when show_logs is True

        dg = AEGenerator(iX=X, iy=cy, dist=dist, noise=noise,
                         batch_size=batch_size, scale=aug_scale,
                         show_logs=False, gt=y,
                         output_channel=1 if model == 'MobileNet' else 3)

    """

    def __init__(self, imgs, pseudo_label, dist, ground_label, input_shape,
                 batch_size=64, num_classes_per_batch=16,num_samples_per_class=4, show_logs=False, prob=1.0, validation=False):

        self.batch_size  = batch_size
        self.x = imgs
        self.py = pseudo_label
        self.gy = ground_label

        # Probability of augmentation
        self.prob = prob

        if self.py.shape[-1] == 1:
            self.py = np.squeeze(self.py)
        if self.gy.shape[-1] == 1:
            self.gy = np.squeeze(self.gy)

        assert len(input_shape) == 3, 'Input shape must be 3 dimentional'
        self.input_shape = input_shape

        self.total = len(self.x)
        self.dist = dist

        self.num_classes_per_batch = num_classes_per_batch
        self.num_samples_per_class = num_samples_per_class

        # Generates label : indexes_where_label_found
        ulabels = np.unique(self.py)
        self.class_index = dict([(label, list(np.where(self.py == label)[0])) \
                                 for label in ulabels])

        self.indexes     = np.arange(self.total)
        self.total_batch = self.total // self.batch_size
        self.classes = len(ulabels)

        self.rand = random.Random(12)
        random.seed(12)

        # Show logs
        self.show_logs = show_logs
        self.validation = validation
        self.log = {}
        self.on_epoch_end()



    def __len__(self):
        """ Denotes the number of batches per epoch """
        return self.total_batch


    def _print_logs(self):
        for a, b in self.log.items():
            print(a, ':', b)


    def on_epoch_end(self):
        """ Updates indexes after each epoch """
        #np.random.shuffle(self.indexes)

        # Shuffle class indices for all data
        for key in self.class_index.keys():
          np.random.shuffle(self.class_index[key])

        p = 0
        get_class = -1
        for batch_index in range(self.__len__()):
            # Selecting half data of same class
            for i in range(self.batch_size//self.num_samples_per_class):
                get_class = self._take_label(get_class)
                for j in range(self.batch_size//self.num_classes_per_batch):
                    self.indexes[p] = self.rand.choice(self.class_index[get_class])
                    p += 1

                # # Selecting half data of same class, different than the previous
                # get_class = self._take_label(get_class)
                # for j in range(self.batch_size//2):
                #     self.indexes[p] = self.rand.choice(self.class_index[get_class])
                #     p += 1

            np.random.shuffle(self.indexes[p-self.batch_size:p])


        if self.show_logs:
            self._print_logs()
            self.log['canNotLink_error'] = 0
            self.log['time'] = 0



    def _make_choice(self, p=None):
        w = [0.5, 0.5]
        if p is not None:
            w = [p, 1-p]
        return self.rand.choices([True, False], weights=w)[0]


    def _take_label(self, avoid=-1):
        take = avoid
        while take == avoid:
          take = self.rand.choice(list(self.class_index.keys()))
        return take



    def read_image(self, image_id):
        """Reads an image from disk, given image_id
            Randomly crops the image w.r.t. input_shape
        """
        im = Image.open(self.x[image_id])
        # Resize the image w.r.t. aspect ratio
        im = AEGenerator.image_resize(im, height=self.input_shape[0])
        im = np.asarray(im, dtype=np.uint8)


        # if self.validation:
        #     val_im = self.line_image_mean(im)
        #     return val_im

        w_pos_l = np.random.randint(low=0, high=max(im.shape[1]-self.input_shape[1]-1, 1))
        return im[..., w_pos_l:w_pos_l+self.input_shape[1]]


    def _augment(self, imgs):
        """ Inputs some image indices,
            Returns the augmented images
            Input_dim : [2*batch_size, ]
        """
        aug_imgs = np.zeros((len(imgs), ) + self.input_shape, dtype=np.uint8)

        #print('Imgs:', imgs.shape)
        for i in range(len(imgs)):
            aug_imgs[i] = self.aug_func(image=imgs[i])['image']
            # print('Augmented')
            # plt.imshow(aug_imgs[i], cmap='gray')
            # plt.show()

        return aug_imgs


    def __getitem__(self, batch_index):
        """ Generate one batch of data """
        idx, y = self._genIndexes(batch_index)
        #img_dirs = self.x[idx]
        # print(self.class_index)
        # Reading the image files

        imgs = np.ones((self.batch_size, ) + self.input_shape, dtype=np.uint8)
        for i in range(self.batch_size):
            # print(idx[i])
            tmp_img = self.x[idx[i]]
            imgs[i] = tmp_img



        return imgs,y


    def _genIndexes(self, index):
        idx = np.zeros((self.batch_size), dtype=np.int16)
        idx = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        constraint = self.py[idx]
        # print(constraint)


        # for i in range(self.batch_size):
        #     plabel = self.py[idx[0][i]]
        #     take = idx[0][i]

        #     # Generating positive
        #     idx[1][i] = self.rand.choice(self.class_index[plabel])
        #     # Generating negative
        #         # Taking random pair
        #     while self.py[take] == plabel:
        #           take = self.rand.choice(self.indexes)
        #     constraint[i] = self.dist

        #     if self.show_logs and self.gy[idx[0][i]] == self.gy[take]:
        #         self.log['canNotLink_error'] += 1

        #     idx[2][i] = take

        return idx, constraint


In [ ]:
class Distance(tf.keras.layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        # ap_distance = tf.expand_dims(tf.math.sqrt(tf.math.reduce_sum(tf.math.square(anchor-positive),
        #                                                              axis=1)),
                                    #  axis=-1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        # an_distance = tf.expand_dims(tf.math.sqrt(tf.math.reduce_sum(tf.math.square(anchor-negative),
        #                                                              axis=1)),
                                    #  axis=-1)
        return (ap_distance, an_distance)


def triplet_loss_func(y_true, y_pred, mergin=0.3):

	ap_distance, an_distance = y_pred[0], y_pred[1]

	loss_1 = tf.add(tf.subtract(ap_distance,an_distance), mergin)
	loss = tf.reduce_sum(tf.maximum(loss_1, 0.0))

	return loss+0.03


def buildAE(input_shape, dims, alpha=1, dis=100, topmodel = 'MobileNet',
            scale=None, setup=0, dropout=0.001, decay=False):

    inp1 = Input(input_shape, name='input1')
    input1 = tf.keras.layers.Lambda(lambda x: (x/255.0))(inp1)
    # inp2 = Input(input_shape, name='input2')
    # input2 = tf.keras.layers.Lambda(lambda x: (x/255.0))(inp2)
    # inp3 = Input(input_shape, name='input3')
    # input3 = tf.keras.layers.Lambda(lambda x: (x/255.0))(inp3)


    topmodel = eval(f"{topmodel}(input_shape={input_shape}," +
                        "include_top=False, weights='imagenet')")

    m = Sequential([topmodel,
                    Flatten(),
                    Dense(dims)], name='AutoEmbedder')
    #m = SincNet(input_shape, dims, softmax=True)

    out1 = m(input1)
    # out2 = m(input2)
    # out3 = m(input3)
    out1 = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(out1)



    # Calculating distance
    # out = Distance()(out1, out2,out3)

    # Initializing model
    # model = Model([inp1,inp2,inp3], out, name="AE_train")
    model = Model(inp1,out1,name='AE_train')
    # Using default adam optimizer with mean square error
    # if decay:
    #     print('Decay loaded')
    #     lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    #                    initial_learning_rate=0.1, decay_steps=50,
    #                    decay_rate=0.0005)

    # Optimizer default:
    # tf.keras.optimizers.Adam(learning_rate=0.0007 if decay is False else lr_schedule)
    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0005),
                  loss = tfa.losses.TripletSemiHardLoss())
    AE = Model(inp1, out1, name="AutoEmbedder")

    return model, AE


previous callback


In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.cluster import Birch
from sklearn.metrics import confusion_matrix
from scipy.optimize import linear_sum_assignment

def accuracy(true_row_labels, predicted_row_labels):
    """Get the best accuracy.
    Parameters
    ----------
    true_row_labels: array-like
        The true row labels, given as external information
    predicted_row_labels: array-like
        The row labels predicted by the model
    Returns
    -------
    float
        Best value of accuracy
    """
    def _make_cost_m(cm):
        s = np.max(cm)
        return (- cm + s)

    cm = confusion_matrix(true_row_labels, predicted_row_labels)
    rows, columns = linear_sum_assignment(_make_cost_m(cm))
    total = 0
    for row, column in zip(rows, columns):
        value = cm[row][column]
        total += value

    return (total * 1. / np.sum(cm))
class MyLogger(tf.keras.callbacks.Callback):
    def __init__(self, n=1, validation_data=None,savepath=None,save_model=True,AE=None):
        self.n = n   # print loss & acc every n epochs
        if validation_data is not None:
            self.x_val, self.py_val ,self.gy_val= validation_data
            self.py_classes = len(np.unique(self.py_val))
            self.gy_classes = len(np.unique(self.gy_val))
        self.start_time = time.time()
        self.start_epoch = 0
        self.savepath=savepath
        self.save_model = save_model
        self.AE = AE

        self.minLoss = 99999
        self.max_K_ACC = 0
        self.max_K_NMI = 0
        self.max_K_ARI = 0

        self.max_EM_ACC = 0
        self.max_EM_NMI = 0
        self.max_EM_ARI = 0

        self.max_B_ACC = 0
        self.max_B_NMI = 0
        self.max_B_ARI = 0

        self.savelog = {'Epoch': [], 'loss' : [],
                        'K-means_PY_ACC':[], 'K-means_PY_NMI':[], 'K-means_PY_ARI':[], 'K-means_GY_ACC':[], 'K-means_GY_NMI':[], 'K-means_GY_ARI':[],
                        'EM_PY_ACC':[], 'EM_PY_NMI':[], 'EM_PY_ARI':[], 'EM_GY_ACC':[], 'EM_GY_NMI':[], 'EM_GY_ARI':[],}
                        #'Birch_PY_ACC':[], 'Birch_PY_NMI':[], 'Birch_PY_ARI':[], 'Birch_GY_ACC':[], 'Birch_GY_NMI':[], 'Birch_GY_ARI':[]}



        if self.savepath != None:
            self.logpath = os.path.join(self.savepath, 'log.pickle')
            self.opti_path = os.path.join(self.savepath, 'optimizer.pkl')
            self.modelpath = os.path.join(self.savepath, 'model_weights.h5')
        # Creating save paths
        if self.savepath != None and os.path.exists(self.savepath) == False:
            os.makedirs(self.savepath)
        # Loading previous data if found
        if self.savepath != None and os.path.exists(self.logpath):
            with open(self.logpath, 'rb') as f:
                self.savelog = pickle.load(f)
            print('Previous data loaded, starting epoch:', self.savelog['Epoch'][-1])
            self.start_epoch = self.savelog['Epoch'][-1]
            self.minLoss = min(self.savelog['loss'])


            self.max_K_ACC = max(self.savelog['K-means_GY_ACC'])
            self.max_K_NMI = max(self.savelog['K-means_GY_NMI'])
            self.max_K_ARI = max(self.savelog['K-means_GY_ARI'])

            self.max_EM_ACC = max(self.savelog['EM_GY_ACC'])
            self.max_EM_NMI = max(self.savelog['EM_GY_NMI'])
            self.max_EM_ARI = max(self.savelog['EM_GY_ARI'])

            # self.max_B_ACC = max(self.savelog['Birch_GY_ACC'])
            # self.max_B_NMI = max(self.savelog['Birch_GY_NMI'])
            # self.max_B_ARI = max(self.savelog['Birch_GY_ARI'])




    def on_epoch_end(self, epoch, logs={}):

        self.start_epoch +=1
        if self.start_epoch % self.n != 0 and self.start_epoch != 1: return


        self.start_time = time.time()

        t_k_means_acc, t_k_means_nmi, t_k_means_ari = self._KmeansAcc(self.py_classes,self.py_val)
        v_k_means_acc, v_k_means_nmi, v_k_means_ari = self._KmeansAcc(self.gy_classes,self.gy_val)

        t_em_acc, t_em_nmi, t_em_ari = self._EMG(self.py_classes,self.py_val)
        v_em_acc, v_em_nmi, v_em_ari = self._EMG(self.gy_classes,self.gy_val)

        # t_birch_acc, t_birch_nmi, t_birch_ari = self._Birch(self.py_classes,self.py_val)
        # v_birch_acc, v_birch_nmi, v_birch_ari = self._Birch(self.gy_classes,self.gy_val)

        ep_time = time.time() - self.start_time
        print(f"\rEpoch {self.start_epoch}: K-means_PY_NMI {t_k_means_nmi:.5f} K-means_PY_ACC {t_k_means_acc:.5f} K-means_PY_ARI {t_k_means_ari:.5f} Time:{ep_time:.1f}",
              end='\n')
        print(f"\rEpoch {self.start_epoch}: K-means_GY_NMI {v_k_means_nmi:.5f} K-means_GY_ACC {v_k_means_acc:.5f} K-means_GY_ARI {v_k_means_ari:.5f} Time:{ep_time:.1f}",
              end='\n')
        print(f"\rEpoch {self.start_epoch}: EM_PY_NMI {t_em_nmi:.5f} EM_PY_ACC {t_em_acc:.5f} EM_PY_ARI {t_em_ari:.5f} Time:{ep_time:.1f}",
              end='\n')
        print(f"\rEpoch {self.start_epoch}: EM_GY_NMI {v_em_nmi:.5f} EM_GY_ACC {v_em_acc:.5f} EM_GY_ARI {v_em_ari:.5f} Time:{ep_time:.1f}",
              end='\n')
        # print(f"\rEpoch {self.start_epoch}: Birch_PY_NMI {t_birch_nmi:.5f} Birch_PY_ACC {t_birch_acc:.5f} Birch_PY_ARI {t_birch_ari:.5f} Time:{ep_time:.1f}",
        #       end='\n')
        # print(f"\rEpoch {self.start_epoch}: Birch_GY_NMI {v_birch_nmi:.5f} Birch_GY_ACC {v_birch_acc:.5f} Birch_GY_ARI {v_birch_ari:.5f} Time:{ep_time:.1f}",
        #       end='\n')
        self._saveLog(self.start_epoch, logs['loss'],
                      t_k_means_acc, t_k_means_nmi, t_k_means_ari, v_k_means_acc, v_k_means_nmi, v_k_means_ari,
                      t_em_acc, t_em_nmi, t_em_ari, v_em_acc, v_em_nmi, v_em_ari,)
                      # t_birch_acc, t_birch_nmi, t_birch_ari, v_birch_acc, v_birch_nmi, v_birch_ari)

        if self.savepath != None:# and logs['loss'] < self.minLoss:
            print(f'Saving model, prev :{self.minLoss:.2f}')
            self.minLoss = logs['loss']
            with open(self.logpath, 'wb') as f:
                pickle.dump(self.savelog, f)
            if self.save_model:
                #tf.keras.models.save_model(self.model, self.modelpath)
                self.model.save_weights(filepath=self.modelpath, overwrite=True, )

                symbolic_weights = getattr(self.model.optimizer, 'weights')
                weight_values = K.batch_get_value(symbolic_weights)
                with open(self.opti_path, 'wb') as f:
                    pickle.dump(weight_values, f)

        # elif self.savepath != None:
        #     with open(self.logpath, 'wb') as f:
        #         pickle.dump(self.savelog, f)

        self.max_K_ACC = max(v_k_means_acc, self.max_K_ACC)
        self.max_K_ARI = max(v_k_means_ari, self.max_K_ARI)
        self.max_K_nmi = max(v_k_means_nmi, self.max_K_NMI)

        self.max_EM_ACC = max(v_em_acc, self.max_EM_ACC)
        self.max_EM_ARI = max(v_em_ari, self.max_EM_ARI)
        self.max_EM_nmi = max(v_em_nmi, self.max_EM_NMI)

        # self.max_B_ACC = max(v_birch_acc, self.max_B_ACC)
        # self.max_B_ARI = max(v_birch_ari, self.max_B_ARI)
        # self.max_B_nmi = max(v_birch_nmi, self.max_B_NMI)


    def _saveLog(self, epoch, loss,t_k_means_acc, t_k_means_nmi, t_k_means_ari, v_k_means_acc, v_k_means_nmi, v_k_means_ari, t_em_acc, t_em_nmi, t_em_ari, v_em_acc, v_em_nmi, v_em_ari):#, t_birch_acc, t_birch_nmi, t_birch_ari, v_birch_acc, v_birch_nmi, v_birch_ari):
        self.savelog['Epoch'].append(epoch)
        self.savelog['loss'].append(loss)
        self.savelog['K-means_PY_ACC'].append(t_k_means_acc)
        self.savelog['K-means_PY_NMI'].append(t_k_means_nmi)
        self.savelog['K-means_PY_ARI'].append(t_k_means_ari)
        self.savelog['K-means_GY_ACC'].append(v_k_means_acc)
        self.savelog['K-means_GY_NMI'].append(v_k_means_nmi)
        self.savelog['K-means_GY_ARI'].append(v_k_means_ari)

        self.savelog['EM_PY_ACC'].append(t_em_acc)
        self.savelog['EM_PY_NMI'].append(t_em_nmi)
        self.savelog['EM_PY_ARI'].append(t_em_ari)
        self.savelog['EM_GY_ACC'].append(v_em_acc)
        self.savelog['EM_GY_NMI'].append(v_em_nmi)
        self.savelog['EM_GY_ARI'].append(v_em_ari)

        # self.savelog['Birch_PY_ACC'].append(t_birch_acc)
        # self.savelog['Birch_PY_NMI'].append(t_birch_nmi)
        # self.savelog['Birch_PY_ARI'].append(t_birch_ari)
        # self.savelog['Birch_GY_ACC'].append(v_birch_acc)
        # self.savelog['Birch_GY_NMI'].append(v_birch_nmi)
        # self.savelog['Birch_GY_ARI'].append(v_birch_ari)


    def _EMG(self,classes,y_val):

        EM = GaussianMixture(n_components=classes)
        # AE = self.model.layers[2]
        rindex = random.sample(range(0, y_val.shape[0]), classes*5)
        outs = self.AE.predict(self.x_val[rindex])
        y_pred = EM.fit_predict(outs)
        acc = accuracy(y_val[rindex],y_pred)
        nmi = normalized_mutual_info_score(y_val[rindex], y_pred)
        ari = adjusted_rand_score(y_val[rindex], y_pred)

        return (acc, nmi, ari)

    def _Birch(self,classes,y_val):
        birch = Birch(n_clusters=classes)
        AE = self.model.layers[2]
        outs = AE.predict(self.x_val)
        y_pred = birch.fit_predict(outs)
        acc = accuracy(y_val,y_pred)
        nmi = normalized_mutual_info_score(y_val, y_pred)
        ari = adjusted_rand_score(y_val, y_pred)

        return (acc, nmi, ari)


    def _KmeansAcc(self,classes,y_val):
        kmeans = KMeans(n_clusters=classes, n_init=10)
        #AE = self.model.layers[2]
        rindex = random.sample(range(0, y_val.shape[0]), classes*5)
        outs = self.AE.predict(self.x_val[rindex])
        y_pred = kmeans.fit_predict(outs)
        acc = accuracy(y_val[rindex], y_pred)
        nmi = normalized_mutual_info_score(y_val[rindex], y_pred)
        ari = adjusted_rand_score(y_val[rindex], y_pred)
        #top_5 = top_k_accuracy_score(self.y_val[rindex],y_pred,k=5)

        return (acc, nmi, ari)

In [ ]:
model='DenseNet121'
dist=100
input_shape = (112,224,3)
dims=16
batch_size=64
epochs=200
num_classes_per_batch = 16
num_samples_per_class =4

writer = 25
line = 5
n=5

base_dir = '/content/drive/MyDrive/UWriter_final/Triplet/logs'
load_pretrain=False
pretrain_path=None

In [ ]:



# def trainModel(model, X, cy, y, input_shape = (112,224,3), dist=100, dims=3, setup=2,
#                batch_size=64, epochs=3000, writer="25",line=5,
#                load_pretrain=False, base_dir=base_dir,
#                pretrain_path=None):
'''
    This would check if previously ran model exists, and load it in model and
    AE. This function would return MyLogger callback object and the starting
    epoch for the model. Necessary savepath parameters would be declared in this
    function as parameter.

    Possible Naming Scheme: Model_dist_dims_setup_scale_frame_[T,L,B]_MIN_SPKRs_CLIM
    T : TIMIT
    L : LibriSpeech
    B : Bengali ASR
    dist  : Distance between clusters (known as AutoEmbedder alpha parameter)
    dims  : The output dimention of AutoEmbedder
    frame : Frame size (input frame size)
    scale : The scaling feature of speech (spectrogram, mfcc)
    SPKRS : Number of speakers
    CLIM  : Maximum Inter-cluster-linkage
    MIN   : Minutes of speech per-speaker
    '''

strat = tf.distribute.MirroredStrategy()
global AE
tf.keras.backend.clear_session()
nmodel = f"{model}" if model == 'MobileNet' else f"{model}"
pname = f"{nmodel}_D{dims}_W{writer}_"

loaded = False
save_dir = os.path.join(base_dir, pname)
opti_dir = os.path.join(save_dir,'optimizer.pkl')

dg = AEGenerator(imgs=X, pseudo_label=py,
                    dist=100, ground_label=gy, input_shape=(112, 224, 3),
                    batch_size=batch_size, show_logs=True, validation=False,
                    # Setting augmentation probability
                    prob=0)

    #if not os.path.exists(save_dir):
    #    os.makedirs(save_dir)
if os.path.exists(os.path.join(save_dir, 'model_weights.h5')):
    try:
        print('Previous weights found, building model')
        # model, AE = load_AE(os.path.join(save_dir, 'model'),
        #
        with strat.scope():
            model, AE = buildAE(input_shape, dims=dims,
                                dis=100, topmodel=model)
            print('Loading weights')
            model.load_weights(os.path.join(save_dir, 'model_weights.h5'))
        loaded = True
    except Exception as e:
        print(str(e))
        pass

    # model._make_train_function()
    print('Previous optimizer found,loading optimizer')
    grad_vars = model.trainable_weights
    zero_grads = [tf.zeros_like(w) for w in grad_vars]
    model.optimizer.apply_gradients(zip(zero_grads, grad_vars))
    with open(opti_dir, 'rb') as f:
      weight_values = pickle.load(f)
    model.optimizer.set_weights(weight_values)



# if not loaded and load_pretrain and pretrain_path is not None:
#     print('Loading pre-trained model')
#     #model, AE = load_AE(pretrain_path, dg.output_shape[1:])
#     model, AE = buildAE(input_shape, dims=dims,
#                             dis=100, topmodel=model)
#     loaded = True

#     model.load_weights(os.path.join(save_dir, 'model_weights.h5'))
#     #model._make_train_function()
#     with open(opti_dir, 'rb') as f:
#         weight_values = pickle.load(f)
#     model.optimizer.set_weights(weight_values)


if not loaded:
    print('Initializing new model')
    with strat.scope():
        model, AE = buildAE(input_shape, dims=dims,
                            dis=100, topmodel=model)
    loaded = True


print(X.shape, py.shape, gy.shape)

log = MyLogger(n,validation_data=(X,py,gy),savepath=save_dir,
               save_model=True,AE=AE)

model.summary()
model.get_layer('AutoEmbedder').summary()
    #time.sleep(1)
    #AE.summary()

print('Starting model training')
model.fit(dg, epochs=epochs+1, verbose=1, callbacks=[log],)

Previous weights found, building model
29097984/29084464 [==============================] - 0s 0us/step
Loading weights
Previous optimizer found,loading optimizer
(1401, 112, 224, 3) (1401,) (1401,)
Previous data loaded, starting epoch: 595
Model: "AE_train"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input1 (InputLayer)         [(None, 112, 224, 3)]     0         
                                                                 
 lambda (Lambda)             (None, 112, 224, 3)       0         
                                                                 
 AutoEmbedder (Sequential)   (None, 16)                7381584   
                                                                 
 lambda_1 (Lambda)           (None, 16)                0         
                                                                 
Total params: 7,381,584
Trainable params: 7,297,936
Non-trainable params: 83,648


In [ ]:
def plotter(savelog,average,algorithm=None):
        clear_output(wait=True)
        plt.figure(figsize=(12.8, 4.8))

        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        # dashes are the train data
        plt.plot(savelog['Epoch'], (savelog['ACC']),
                '--', label='tACC', c='purple')
        plt.plot(savelog['Epoch'], (savelog['NMI']),
                '--', label='tNMI', c='orange')
        plt.plot(savelog['Epoch'], (savelog['ARI']),
                '--', label='tARI', c='c')
        # solids are validation data
        plt.plot(savelog['Epoch'], (savelog['val_ACC']),
                 label='ACC', c='purple')
        plt.plot(savelog['Epoch'], (savelog['val_NMI']),
                 label='NMI', c='orange')
        plt.plot(savelog['Epoch'], (savelog['val_ARI']),
                 label='ARI', c='c')
        plt.ylabel('Score')
        plt.xlabel('Epoch')

        plt.grid()
        plt.minorticks_on()
        plt.grid(b=True, which='minor', linestyle='--', alpha=0.25)
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.13),
                   fancybox=True, shadow=False, ncol=5)